In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Paths to data files
train_csv = os.path.expanduser("data/df_cleaned_20250617.csv")
test_csv  = "data/st_vgp_pois_constmean_t400r300_test_predictions.csv"

# Load and preprocess training data
df_train = pd.read_csv(train_csv, parse_dates=['timestamp'])
df_train = df_train.dropna(subset=['ground_truth'])  # only actual observations

# Load prediction results
df_test = pd.read_csv(test_csv, parse_dates=['timestamp'])

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_72116/1167199628.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_test = pd.read_csv(test_csv, parse_dates=['timestamp'])


In [107]:
# Convert the seconds‐since‐epoch column to datetime
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'], unit='s').dt.date

# Make sure both dataframes have the same date format
df_train['timestamp'] = pd.to_datetime(df_train['timestamp']).dt.date

# Rename columns for consistency
df_train.rename(columns={'ground_truth': 'count'}, inplace=True)
df_test.rename(columns={'pred_mean_Y': 'count','pred_var_Y':'var','rate_median':'median','rate_lower95':'lower95','rate_upper95':'upper95','rate_lower90':'lower90','rate_upper90':'upper90'}, inplace=True)

# Add a 'category' column to distinguish between training and prediction data
df_train['category'] = 'train'
df_test['category'] = 'prediction'

# Create a empty columns to combine two dataframes
df_train['median'] = np.nan
df_train['var'] = np.nan
df_train['lower95'] = np.nan
df_train['upper95'] = np.nan
df_train['lower90'] = np.nan
df_train['upper90'] = np.nan

# Combine the two dataframes
df_combined = pd.concat([df_train, df_test], ignore_index=True)

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_72116/2344571355.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_test['timestamp'] = pd.to_datetime(df_test['timestamp'], unit='s').dt.date


In [111]:
# Needed columns
needed_columns = ['timestamp', 'count', 'median', 'var', 'lower95', 'upper95', 'lower90', 'upper90', 'category']
df_combined = df_combined[needed_columns]

# Fill Nan values with 0
df_combined.fillna(0, inplace=True)

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_72116/385323642.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined.fillna(0, inplace=True)


In [149]:
# Load ground-truch sf data
df_sf = pd.read_csv('data/sf_tent.csv')

# Drop rows with NaN in 'date' column
df_sf = df_sf.dropna(subset=['date'], axis=0)

# Create timestamp column in df_sf using year, month, and day
df_sf.rename(columns={'date': 'day'}, inplace=True)
df_sf['timestamp'] = pd.to_datetime(df_sf[['year', 'month', 'day']])

df_sf

,year,month,day,tents,structure,sites,tent_struc,timestamp
3,2019,4,23.0,207.0,173.0,442.0,380,2019-04-23
4,2019,4,24.0,207.0,173.0,442.0,380,2019-04-24
5,2019,7,23.0,264.0,187.0,437.0,451,2019-07-23
6,2019,7,24.0,264.0,187.0,437.0,451,2019-07-24
7,2019,10,23.0,272.0,175.0,501.0,447,2019-10-23
8,2020,1,22.0,459.0,190.0,601.0,649,2020-01-22
9,2020,4,20.0,897.0,211.0,427.0,1108,2020-04-20
10,2020,4,21.0,897.0,211.0,427.0,1108,2020-04-21
11,2020,4,22.0,897.0,211.0,427.0,1108,2020-04-22
12,2020,7,29.0,790.0,213.0,426.0,1003,2020-07-29


## Aggregation

In [151]:
# Monte Carlo aggregation to city‐daily totals with P(Y>0) thresholding
S          = 500
p_thresh   = 0.70   # drop any box whose P(Y>0) < 0.10
daily_out  = []
n_days     = df_combined['timestamp'].nunique()

# z‐score for 95% CI
z95 = 1.96
z90 = 1.645

from tqdm import tqdm
for day, grp in tqdm(df_combined.groupby('timestamp'),
                     total=n_days,
                     desc="Aggregating city totals"):
    # pull out your per-box summaries
    med   = grp['median'].values    # median λ_i
    l95   = grp['lower95'].values
    # compute P(Y_i > 0) ≈ 1 - exp(-med)
    p_any = 1 - np.exp(-med)
    
    # keep only boxes likely to have tents
    keep_mask = p_any >= p_thresh
    if not keep_mask.any():
        # no boxes pass the threshold → zero tents
        daily_out.append({
            'timestamp':  day,
            'mean_total': 0.0,
            'median_total': 0.0,
            'lower95': 0.0,
            'upper95': 0.0,
            'lower90': 0.0,
            'upper90': 0.0,
        })
        continue
    
    med_filt = med[keep_mask]
    l95_filt = l95[keep_mask]
    nbox     = med_filt.size
    
    # reconstruct log-normal latent params
    mu_f    = np.log(med_filt)
    sigma_f = (np.log(med_filt) - np.log(l95_filt)) / z95
    
    # Monte Carlo draws
    f_samps = np.random.normal(
        loc=mu_f[None, :],
        scale=sigma_f[None, :],
        size=(S, nbox)
    )
    lam_samps = np.exp(f_samps)
    # optional hard cap
    lam_samps = np.minimum(lam_samps, 1e3)
    
    # draw Poisson counts and sum
    y_samps   = np.random.poisson(lam_samps)
    city_samps = y_samps.sum(axis=1)
    
    # store summary
    daily_out.append({
        'timestamp':    day,
        'mean_total':   city_samps.mean(),
        'median_total': np.median(city_samps),
        'lower95':      np.percentile(city_samps, 2.5),
        'upper95':      np.percentile(city_samps, 97.5),
        'lower90':      np.percentile(city_samps, 5.0),
        'upper90':      np.percentile(city_samps, 95.0),
    })

# assemble final daily DataFrame
df_daily_07 = (
    pd.DataFrame(daily_out)
      .sort_values('timestamp')
      .reset_index(drop=True)
)
print(df_daily_07.head())


Aggregating city totals: 100%|██████████| 3074/3074 [01:19<00:00, 38.78it/s] 

    timestamp  mean_total  median_total  lower95  upper95  lower90  upper90
0  2016-01-01     175.012         174.0  152.000  200.000   155.00   195.00
1  2016-01-02     184.494         184.0  158.475  213.000   162.00   208.00
2  2016-01-03     218.854         218.0  192.000  249.000   196.95   243.05
3  2016-01-04     213.872         214.0  187.000  243.525   190.00   237.05
4  2016-01-05     190.796         190.0  167.000  220.000   170.95   215.00


In [188]:
# Make sure the timestamp is in datetime format
df_daily_07['timestamp'] = pd.to_datetime(df_daily_07['timestamp'])
df_sf['timestamp'] = pd.to_datetime(df_sf['timestamp'])

# Calculate RMSE between daily totals and ground truth
pred_values = df_daily_07[df_daily_07['timestamp'].isin(df_sf['timestamp'])]['mean_total'].values
gt_values = df_sf[df_sf['timestamp'] < '2024-06-01']['tents'].values

rmse = np.sqrt(np.mean((pred_values - gt_values) ** 2))
print(f"RMSE between daily totals and ground truth: {rmse:.2f}")


RMSE between daily totals and ground truth: 461.92


In [ ]:
# Monte Carlo aggregation to city‐daily totals with P(Y>0) thresholding
S          = 500
p_thresh   = 0.5   # drop any box whose P(Y>0) < 0.10
daily_out  = []
n_days     = df_combined['timestamp'].nunique()

# z‐score for 95% CI
z95 = 1.96
z90 = 1.645

from tqdm import tqdm
for day, grp in tqdm(df_combined.groupby('timestamp'),
                     total=n_days,
                     desc="Aggregating city totals"):
    # pull out your per-box summaries
    med   = grp['median'].values    # median λ_i
    l95   = grp['lower95'].values
    # compute P(Y_i > 0) ≈ 1 - exp(-med)
    p_any = 1 - np.exp(-med)
    
    # keep only boxes likely to have tents
    keep_mask = p_any >= p_thresh
    if not keep_mask.any():
        # no boxes pass the threshold → zero tents
        daily_out.append({
            'timestamp':  day,
            'mean_total': 0.0,
            'median_total': 0.0,
            'lower95': 0.0,
            'upper95': 0.0,
            'lower90': 0.0,
            'upper90': 0.0,
        })
        continue
    
    med_filt = med[keep_mask]
    l95_filt = l95[keep_mask]
    nbox     = med_filt.size
    
    # reconstruct log-normal latent params
    mu_f    = np.log(med_filt)
    sigma_f = (np.log(med_filt) - np.log(l95_filt)) / z95
    
    # Monte Carlo draws
    f_samps = np.random.normal(
        loc=mu_f[None, :],
        scale=sigma_f[None, :],
        size=(S, nbox)
    )
    lam_samps = np.exp(f_samps)
    # optional hard cap
    lam_samps = np.minimum(lam_samps, 1e3)
    
    # draw Poisson counts and sum
    y_samps   = np.random.poisson(lam_samps)
    city_samps = y_samps.sum(axis=1)
    
    # store summary
    daily_out.append({
        'timestamp':    day,
        'mean_total':   city_samps.mean(),
        'median_total': np.median(city_samps),
        'lower95':      np.percentile(city_samps, 2.5),
        'upper95':      np.percentile(city_samps, 97.5),
        'lower90':      np.percentile(city_samps, 5.0),
        'upper90':      np.percentile(city_samps, 95.0),
    })

# assemble final daily DataFrame
df_daily_05 = (
    pd.DataFrame(daily_out)
      .sort_values('timestamp')
      .reset_index(drop=True)
)
print(df_daily_05.head())

# Make sure the timestamp is in datetime format
df_daily_05['timestamp'] = pd.to_datetime(df_daily_05['timestamp'])
df_sf['timestamp'] = pd.to_datetime(df_sf['timestamp'])

# Calculate RMSE between daily totals and ground truth
pred_values = df_daily_05[df_daily_05['timestamp'].isin(df_sf['timestamp'])]['mean_total'].values
gt_values = df_sf[df_sf['timestamp'] < '2024-06-01']['tents'].values

rmse = np.sqrt(np.mean((pred_values - gt_values) ** 2))
print(f"RMSE between daily totals and ground truth: {rmse:.2f}")

